# Packages

In [1]:
from numpy import *
from prody import *
from matplotlib.pyplot import *
from scipy.stats import hypergeom
import os

In [2]:
from GNMhinges import *

# Build ensembles

In [3]:
currPDB = '3d4s' # change it to your interesting structure with PDB IDs
eachChain = 'A'

# pdb_ids, mappings = getDali_info(currPDB, eachChain, cutoff_len=0.7, cutoff_rmsd=1.0, cutoff_Z=10)
EigVals_similar, averageEigVects_similar, ids_similar, gnms_similar = getModesSimilar(currPDB, eachChain, length=0.8, rmsd=2, Z=10)

@> Submitted Dali search for PDB "3d4sA".
@> http://ekhidna2.biocenter.helsinki.fi/barcosel/tmp//3d4sA/
@> Dali results were fetched in 0.2s.   
@> Obtained 2440 PDB chains from Dali for 3d4sA.
@> 2426 PDBs have been filtered out from 2440 Dali hits (remaining: 14).
@> Retrieving 3d4sA... [  0%]

RMSD less than  2.0 0.8198979 0.35715997
RMSD greater than 1A  4.0229917 2.8395913
# of similar structures is found from Dali 14


@> Retrieving 5x7dA... [ 21%] 7s @> WARNING SEQADV for chain A: database sequence reference not found (5X7D:381)
@> WARNING SEQADV for chain A: database sequence reference not found (5X7D:382)
@> Retrieving 6obaA... [ 64%] 2s@> WARNING SEQADV for chain A: database sequence reference not found (6OBA:394)
@> WARNING SEQADV for chain A: database sequence reference not found (6OBA:395)
@> 14 PDBs were parsed in 4.42s.
@> Starting iterative superposition:             
@> Step #1: RMSD difference = 3.1909e-01
@> Step #2: RMSD difference = 1.5312e-05
@> Iterative superposition completed in 0.01s.
@> Final superposition to calculate transformations.
@> Superposition completed in 0.00 seconds.
@> Ensemble (14 conformations) were built in 0.45s.
@> all GNM modes were calculated for each of the 14 conformations in 1.18s.
@> 438 modes across 14 modesets were matched in 0.33s.


# Define reference structure

In [4]:
GPCR = parsePDB('3d4s', subset='calpha') 
calphas = GPCR.select('ca resnum 1:343') # not whole protein but only 7-transmemebrane domain

@> Connecting wwPDB FTP server RCSB PDB (USA).
@> 3d4s downloaded (3d4s.pdb.gz)
@> PDB download via FTP completed (1 downloaded, 0 failed).
@> 439 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> Secondary structures were assigned to 375 residues.


# GNM motions

In [5]:
ags = parsePDB(ids_similar, subset='ca')
# ags = parsePDB(mergeIDs, subset='ca', chain=Chains) # if you have dimer
dali_ens = buildPDBEnsemble(ags, ref=calphas) 
# dali_ens = buildPDBEnsemble(ags) # if we just used first element of the structure
gnms = calcEnsembleENMs(dali_ens, model='GNM', trim='reduce', n_modes=None)
eigVals = gnms.getEigvals()
averageEigVals = gnms.getEigvals()[0]
eigVects = gnms.getEigvecs()
averageEigVecs = mean(eigVects, axis=0)

@> Retrieving 5x7dA... [ 21%] 3s@> WARNING SEQADV for chain A: database sequence reference not found (5X7D:381)
@> WARNING SEQADV for chain A: database sequence reference not found (5X7D:382)
@> Retrieving 6obaA... [ 64%] 2s@> WARNING SEQADV for chain A: database sequence reference not found (6OBA:394)
@> WARNING SEQADV for chain A: database sequence reference not found (6OBA:395)
@> 14 PDBs were parsed in 3.56s.
@> Starting iterative superposition:             
@> Step #1: RMSD difference = 2.4047e-01
@> Step #2: RMSD difference = 5.1293e-06
@> Iterative superposition completed in 0.01s.
@> Final superposition to calculate transformations.
@> Superposition completed in 0.00 seconds.
@> Ensemble (14 conformations) were built in 0.42s.
@> all GNM modes were calculated for each of the 14 conformations in 0.70s.
@> 278 modes across 14 modesets were matched in 0.21s.


# 33% contribution

In [6]:
currNumModes = getModesGivenThreshold(averageEigVals, 0.33)
currNumModes

3

# Allocate hinges and define binding sites

In [12]:
bindings = [27,38,41,42,45,48,49,52,53,76,77,78,80,81,82,83,85,86,119,122,123,126,134,161,163,167,168,171,172,175,222,225,226,229,244,248,252]
binding = list(set([int(x) for x in bindings]))

In [13]:
Hinges_3modes = getHinges2(averageEigVecs, 3, 15) # using 3 modes
Hinges_3modes_final = trimEnds(Hinges_3modes, [[0, 278]], 20) # trim ends

# p values

In [14]:
protein_length = 279

overlaps_2 = len(binding) + len(Hinges_3modes_final) - len(set(binding + Hinges_3modes_final))
HyperScore_2 = ORA(len(binding), protein_length, len(Hinges_3modes_final), overlaps_2)

print ('# of binding sites is', len(binding))
print ('# of hinge sites for threshold 0.33, overlap, hyper score', len(Hinges_3modes_final), overlaps_2, HyperScore_2)

# of binding sites is 37
# of hinge sites for threshold 0.33, overlap, hyper score 33 11 0.001225404684947029
